In [1]:
!pip uninstall lightgbm --yes
!pip install lightgbm --no-cache-dir


Found existing installation: lightgbm 4.2.0
Uninstalling lightgbm-4.2.0:
  Successfully uninstalled lightgbm-4.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.4 MB/s eta 0:00:0000:0100:010m


In [2]:
import os

# Set environment variables
os.environ['LDFLAGS'] = '-L/opt/homebrew/opt/libomp/lib'
os.environ['CPPFLAGS'] = '-I/opt/homebrew/opt/libomp/include'

# Now you can try importing LightGBM again
import lightgbm as lgb


OSError: dlopen(/Users/franciscofurey/00DataScience/OpenAi/venv/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <AF2B35E2-56EE-3669-88A6-BBD75577D81F> /Users/franciscofurey/00DataScience/OpenAi/venv/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [2]:
!pip3 install -q --upgrade pip
!pip3 install -q pandas numpy matplotlib seaborn openpyxl climateserv requests netCDF4 xarray pyproj statsmodels

In [5]:
import sys
from pathlib import Path

# Calculate the path to the scripts folder relative to the current notebook.
scripts_dir = Path("../../").resolve()

# Add the scripts directory to the sys.path if it's not already there.
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV, cross_val_score

OSError: dlopen(/Users/franciscofurey/00DataScience/OpenAi/venv/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <AF2B35E2-56EE-3669-88A6-BBD75577D81F> /Users/franciscofurey/00DataScience/OpenAi/venv/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [ ]:
# Carga de datos y preparación inicial (igual que tu código)
df = pd.read_csv("../../data/processed_data/wells_data_gambia_for_machine_learning.csv")
unique_values = df['DepthToGroundwater'].unique()
value_to_int = {value: idx for idx, value in enumerate(unique_values)}
df['DepthToGroundwater'] = df['DepthToGroundwater'].map(value_to_int)

# Preparación de los conjuntos de datos (igual que tu código)
unique_ids = df['ID'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
train_df = df[df['ID'].isin(train_ids)]
test_df = df[df['ID'].isin(test_ids)]

X_train = train_df.drop(columns=['GROUNDWATER_LEVEL', 'ID', 'Date'])
y_train = train_df['GROUNDWATER_LEVEL']
X_test = test_df.drop(columns=['GROUNDWATER_LEVEL', 'ID', 'Date'])
y_test = test_df['GROUNDWATER_LEVEL']

# Prepara los grupos para GroupKFold (igual que tu código)
groups = train_df['ID']

# Prepare the LightGBM dataset
d_train = lgb.Dataset(data=X_train, label=y_train, free_raw_data=False)

In [ ]:
# Define parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Define GroupKFold
gkf = GroupKFold(n_splits=5)

# Initialize an empty array to hold feature importances
feature_importances = []

# Train the model using GroupKFold
for fold_id, (train_index, val_index) in enumerate(gkf.split(X_train, y_train, groups=groups)):
    print(f"Training fold {fold_id + 1}")
    d_train_subset = d_train.subset(train_index)
    d_valid_subset = d_train.subset(val_index)

    bst = lgb.train(params, d_train_subset, num_boost_round=1000, valid_sets=[d_valid_subset], early_stopping_rounds=10, verbose_eval=False)
    feature_importances.append(bst.feature_importance())

    y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Fold {fold_id + 1} MSE: {mse}")

# Average feature importances over all folds
average_feature_importances = sum(feature_importances) / len(feature_importances)

In [ ]:
# Apply best parameters
nystroem = Nystroem(
    kernel=best_params['nystroem__kernel'],
    gamma=best_params['nystroem__gamma'],
    n_components=best_params['nystroem__n_components'],
    random_state=1
)

linear_svr = LinearSVR(
    C=best_params['svr__C'],
    max_iter=10000,
    random_state=42
)

# Define a new, more focused parameter grid for fine-tuning
param_grid_fine = {
    'nystroem__gamma': [best_params['nystroem__gamma'] * factor for factor in [0.8, 1, 1.2]],
    'svr__C': [best_params['svr__C'] * factor for factor in [0.8, 1, 1.2]]
}

# Update the pipeline with the best parameters
best_model.set_params(**best_params)

# New Grid search for fine-tuning
grid_search_fine = GridSearchCV(
    estimator=best_model,  # Use the best model from the initial grid search
    param_grid=param_grid_fine,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Fit the model with groups for the fine-tuning grid search
grid_search_fine.fit(X=X_train, y=y_train, groups=groups)

# Extract new best parameters
new_best_params = grid_search_fine.best_params_
print("New best parameters:", new_best_params)

In [ ]:
# Using the best parameters from your grid searches
final_nystroem = Nystroem(
    kernel='sigmoid',  # From best parameters
    gamma=0.08,  # From new best parameters
    n_components=100,  # From best parameters
    random_state=1
)

final_linear_svr = LinearSVR(
    C=0.12,  # From new best parameters
    max_iter=10000,
    random_state=42
)

# Set up the final pipeline with the selected parameters
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nystroem', final_nystroem),
    ('svr', final_linear_svr)
])

# Train the model on the entire training dataset
final_pipeline.fit(X_train, y_train)

# Optionally, evaluate the model on the test dataset
test_score = final_pipeline.score(X_test, y_test)
print(f"Test Score: {test_score}")

# Use the model to make predictions
predictions = final_pipeline.predict(X_test)

# Calculate MSE, MAE, and R²
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Coefficient of Determination (R²): {r2}")